In [1]:
import os
import pandas as pd
import glob

# import seaborn as sns
# import ast
import re
import sys

import cv2

import numpy as np
from PIL import Image

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torch import optim
import torchvision
from torchvision import transforms
import torchvision.models as models
import torchvision.transforms as T
import torchvision.transforms.functional as TF

from tqdm import tqdm

import matplotlib.pyplot as plt
%matplotlib inline

yolo_path = "../third/yolov7_custom"
sys.path.append(yolo_path)


device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

In [2]:
from datetime import datetime

task_code = datetime.now().strftime("%H%M%S")

In [3]:
mask_model_ver="132508"
IMAGE_SIZE=640

DATA_DIR = "/root/autodl-tmp/cervical_spine/"
IMAGES_DIR = os.path.join(DATA_DIR, 'train_axial_images_jpeg95_croped_132508')
MASK_DIR = os.path.join(DATA_DIR, 'segmentation_axial_results_132508')

# weights = "/root/cervical_spine/third/yolov7_custom/runs/train/yolov7-custom9/weights/last.pt"
# weights = "/root/autodl-tmp/cervical_spine/checkpoint/yolov7-A100-best.pt"

In [4]:
train_df = pd.read_csv(os.path.join(DATA_DIR, 'train.csv')).set_index('StudyInstanceUID')
train_df.head()

,patient_overall,C1,C2,C3,C4,C5,C6,C7
StudyInstanceUID,,,,,,,,
1.2.826.0.1.3680043.6200,1,1,1,0,0,0,0,0
1.2.826.0.1.3680043.27262,1,0,1,0,0,0,0,0
1.2.826.0.1.3680043.21561,1,0,1,0,0,0,0,0
1.2.826.0.1.3680043.12351,0,0,0,0,0,0,0,0
1.2.826.0.1.3680043.1363,1,0,0,0,0,1,0,0


In [5]:
total_boundary_df = pd.read_csv(os.path.join(DATA_DIR, f'infered_boundary_{mask_model_ver}_2.csv'))
total_boundary_df['UID_Slice'] = total_boundary_df['StudyInstanceUID'] + '.' + total_boundary_df['Slice'].astype('string')
total_boundary_df = total_boundary_df.set_index('UID_Slice').astype({'Slice': int})
print(len(total_boundary_df.StudyInstanceUID.unique()))
print(len(total_boundary_df))
total_boundary_df.head()

2012
708779


,StudyInstanceUID,Slice,Start,pixel_spacing,xmin,ymin,xmax,ymax
UID_Slice,,,,,,,,
1.2.826.0.1.3680043.10001.0,1.2.826.0.1.3680043.10001,0,0,0.253906,0.0,0.0,512.0,512.0
1.2.826.0.1.3680043.10001.1,1.2.826.0.1.3680043.10001,1,0,0.253906,0.0,0.0,512.0,512.0
1.2.826.0.1.3680043.10001.2,1.2.826.0.1.3680043.10001,2,0,0.253906,0.0,0.0,512.0,512.0
1.2.826.0.1.3680043.10001.3,1.2.826.0.1.3680043.10001,3,0,0.253906,0.0,0.0,512.0,512.0
1.2.826.0.1.3680043.10001.4,1.2.826.0.1.3680043.10001,4,0,0.253906,0.0,0.0,512.0,512.0


In [6]:
from models.experimental import attempt_load

In [7]:
# model = attempt_load(weights, map_location=device)
# model = model.eval()

In [8]:

test_slices = glob.glob(f'{IMAGES_DIR}/1.2.826.0.1.3680043.219*/*')
test_slices = [re.findall(f'{IMAGES_DIR}/(.*)/(.*).jpeg', s)[0] for s in test_slices]

df_test_slices = pd.DataFrame(data=test_slices, columns=['StudyInstanceUID', 'Slice'])
df_test_slices['UID_Slice'] = df_test_slices['StudyInstanceUID'] + '.' + df_test_slices['Slice'].astype('string')
df_test_slices = df_test_slices.set_index('UID_Slice').astype({'Slice': int})
df_test_slices

,StudyInstanceUID,Slice
UID_Slice,,
1.2.826.0.1.3680043.21904.153,1.2.826.0.1.3680043.21904,153
1.2.826.0.1.3680043.21904.0,1.2.826.0.1.3680043.21904,0
1.2.826.0.1.3680043.21904.154,1.2.826.0.1.3680043.21904,154
1.2.826.0.1.3680043.21904.1,1.2.826.0.1.3680043.21904,1
1.2.826.0.1.3680043.21904.155,1.2.826.0.1.3680043.21904,155
...,...,...
1.2.826.0.1.3680043.21997.95,1.2.826.0.1.3680043.21997,95
1.2.826.0.1.3680043.21997.96,1.2.826.0.1.3680043.21997,96
1.2.826.0.1.3680043.21997.97,1.2.826.0.1.3680043.21997,97


In [9]:
df = total_boundary_df.loc[df_test_slices.index].reset_index()
print(len(df))
df.head()

2931


,UID_Slice,StudyInstanceUID,Slice,Start,pixel_spacing,xmin,ymin,xmax,ymax
0,1.2.826.0.1.3680043.21904.153,1.2.826.0.1.3680043.21904,153,0,0.332031,97.411651,173.411651,398.588349,474.588349
1,1.2.826.0.1.3680043.21904.0,1.2.826.0.1.3680043.21904,0,0,0.332031,0.000000,0.000000,512.000000,512.000000
2,1.2.826.0.1.3680043.21904.154,1.2.826.0.1.3680043.21904,154,0,0.332031,127.411651,173.411651,428.588349,474.588349
3,1.2.826.0.1.3680043.21904.1,1.2.826.0.1.3680043.21904,1,0,0.332031,0.000000,0.000000,512.000000,512.000000
4,1.2.826.0.1.3680043.21904.155,1.2.826.0.1.3680043.21904,155,0,0.332031,91.411651,174.411651,392.588349,475.588349


In [10]:
class ImageDataSet(torch.utils.data.Dataset):    
    def __init__(self, df, img_dir, mask_dir, transform):
        super().__init__()
        self.df = df
        self.img_dir = img_dir
        self.mask_dir = mask_dir
        self.transform = transform
        self.len = len(self.df)
        
    def __getitem__(self, i):
        
        s = self.df.iloc[i]
        UID = s.StudyInstanceUID
        
        img_path = os.path.join(self.img_dir, UID, f'{int(s.Slice)}.jpeg')
        img = Image.open(img_path)
        
        mask_path = os.path.join(self.mask_dir, UID, f'{int(s.Slice)}.png')
        mask = Image.open(mask_path)
    
        mask = mask.crop((s.xmin/2, s.ymin/2, s.xmax/2, s.ymax/2))
        
        if self.transform is not None:
            img, mask = self.transform(img, mask)
        
        return img, mask
    
    def __len__(self):
        return self.len

class ImageTransform(nn.Module):
    def __init__(self, image_size=640):
        super().__init__()

        self.image_size = image_size

        self.transform = T.Compose([
            T.Resize((image_size, image_size)),
            T.ToTensor(),
            # T.Normalize(0, 255.),
        ])
        
        self.mask_transform = T.Compose([
            T.Resize((256, 256), interpolation=torchvision.transforms.InterpolationMode.NEAREST),
            T.ToTensor(),
        ])

    def forward(self, x, mask):
        x = self.transform(x)
        mask = self.mask_transform(mask)
        return x, mask
    
tf = ImageTransform(image_size=IMAGE_SIZE)
ds = ImageDataSet(df, IMAGES_DIR, MASK_DIR, tf)

img, mask = ds[200]
print(img.shape, img.min(), img.max())
print(mask.shape, mask.min(), mask.max())

torch.Size([3, 640, 640]) tensor(0.11373) tensor(0.99216)
torch.Size([1, 256, 256]) tensor(0.) tensor(0.14902)


In [11]:
img, mask = ds[100]
_, axs = plt.subplots(1, 2)
axs[0].imshow(img[0, :, :])
axs[1].imshow(mask[0, :, :])

plt.show()

In [12]:
# batch_size = 32
# dl = DataLoader(ds, batch_size=batch_size, shuffle=False, num_workers=min(os.cpu_count(), batch_size))

In [13]:
def pred_det(model, x):
    pred = model(x)[0]
    max_indices = torch.argmax(pred[:, :, 4], dim=1)
    max_values = pred[torch.arange(x.shape[0]), max_indices, :] # N x 6
    
    bboxes, scores = max_values[:, :4], max_values[:, 4]
    bboxes[:, 2] += bboxes[:, 0]
    bboxes[:, 3] += bboxes[:, 1]
    return bboxes, scores
                
def get_bbox_class_list(seg_list, seg_bboxes):
    class_list = []
    for i in range(seg_list.shape[0]):
        class_index = get_bbox_class(seg_list[i, :, :], seg_bboxes[i, :])
        class_list.append(class_index)
        
    return torch.stack(class_list)     


def get_bbox_class(seg, bbox):
    """
    label 은 0.125 의 단위로, 
    seg: H x W
    bbox: [xmin, ymin, xmax, ymax]
    """
    xmin, ymin, xmax, ymax = bbox.int()
    area = seg[ymin:ymax, xmin:xmax]
    
    # print(area)
    result = torch.mean(area[area>0])
    result = torch.round(result / 0.125)
    
    return result

def get_class_score(scores, class_list, eps=1e-2):
    result = scores.new_zeros((scores.shape[0], 8)) + eps
    class_list = torch.nan_to_num(class_list).long()
    result[torch.arange(scores.shape[0]), class_list] = scores
    
    return result


def cal_loss(prob, label):
    
    pos_weight = np.array([14, 2, 2, 2, 2, 2, 2, 2])
    neg_weight = np.array([7, 1, 1, 1, 1, 1, 1, 1])
    
    score = pos_weight * label * np.log(prob) + neg_weight * (1 - label) * np.log(1 - prob)
    
    weight_total = pos_weight * label + neg_weight * (1 - label)
    
    return -score.sum(axis=1) / weight_total.sum(axis=1)


In [14]:
def get_test_df(UIDs):
    """
    UIDs : 1.2.826.0.1.3680043.219*
    """
    test_slices = glob.glob(f'{IMAGES_DIR}/{UIDs}/*')
    test_slices = [re.findall(f'{IMAGES_DIR}/(.*)/(.*).jpeg', s)[0] for s in test_slices]

    df_test_slices = pd.DataFrame(data=test_slices, columns=['StudyInstanceUID', 'Slice'])
    df_test_slices['UID_Slice'] = df_test_slices['StudyInstanceUID'] + '.' + df_test_slices['Slice'].astype('string')
    # df_test_slices = df_test_slices.set_index('UID_Slice').astype({'Slice': int})
    df = total_boundary_df.loc[df_test_slices.UID_Slice].sort_values(['StudyInstanceUID','Slice']).reset_index()

    return df

def get_loader(df, batch_size=32):
    tf = ImageTransform(image_size=640)
    ds = ImageDataSet(df, IMAGES_DIR, MASK_DIR, tf)
    
    dl = DataLoader(ds, batch_size=batch_size, shuffle=False, pin_memory=False, num_workers=min(16, batch_size))
    return dl

In [15]:
weights = "/root/cervical_spine/third/yolov7_custom/runs/train/yolov7-custom4/weights/epoch_099.pt"
model = attempt_load(weights, map_location=device)
model = model.eval()

Fusing layers... 
IDetect.fuse


In [17]:
def predict(dl, batch_size=128):
    
    with torch.no_grad():
        
        predictions = []
        
        for x, mask in tqdm(dl):
            x = x.to(device)
            mask = mask.to(device)
            
            batch_probs = x.new_zeros((x.shape[0], 8)) + 1e-2
            
            active_indices = mask.sum(axis=[1, 2, 3]).nonzero().reshape(-1)
            
            if active_indices.numel() == 0:
                predictions.append(batch_probs)
                continue
                
                
            # infer_result = torch.zeros((x.shape[0], 9))    
            if active_indices.numel() != batch_size:
                x = x[active_indices, :, :, :]
                mask = mask[active_indices, :, :, :]
            
            bboxes, scores = pred_det(model, x)
            
            class_list = get_bbox_class_list(mask[:, 0, :, :], bboxes / (IMAGE_SIZE / 256.) )
            # print(bboxes)
            # print(class_list)
            probs = get_class_score(scores, class_list)
            
            batch_probs[active_indices, :] = probs
#             print(probs)
            predictions.append(batch_probs)
    
        return torch.concat(predictions).cpu().numpy()
    
batch_size = 256
UID = '*'
df = get_test_df(UID)
tf = ImageTransform(image_size=640)
ds = ImageDataSet(df, IMAGES_DIR, MASK_DIR, tf)

dl = DataLoader(ds, batch_size=batch_size, shuffle=False, pin_memory=False, num_workers=16)
predictions = predict(dl, batch_size)

# print("max score slice : ", np.argmax(predictions.max(axis=1), axis=0))

100%|██████████| 2769/2769 [49:11<00:00,  1.07s/it]


In [ ]:
# max_indice = np.argmax(predictions.max(axis=1), axis=0)
# print(f'{UID}.{max_indice}')
# img, mask = ds[max_indice]
# plt.imshow(img[2, :, :], cmap='bone')
# plt.show()

In [18]:
df_effnet_pred = pd.DataFrame(
    data=predictions, columns=['patient_overall'] + [f'C{i}' for i in range(1, 8)]
)
df_test_pred = pd.concat([df, df_effnet_pred], axis=1).sort_values(['StudyInstanceUID', 'Slice'])
df_patient_pred = df_test_pred.groupby('StudyInstanceUID').apply(lambda df: df.max())

clip_value = 1e-3
df_patient_pred[[f'C{i}' for i in range(1, 8)]] = df_patient_pred[[f'C{i}' for i in range(1, 8)]].clip(lower=clip_value, upper=1-clip_value)


df_patient_pred["patient_overall"] = df_patient_pred[[f'C{i}' for i in range(1, 8)]].max(axis=1)
df_patient_pred = df_patient_pred[['patient_overall'] + [f'C{i}' for i in range(1, 8)]]
df_patient_pred

,patient_overall,C1,C2,C3,C4,C5,C6,C7
StudyInstanceUID,,,,,,,,
1.2.826.0.1.3680043.10001,0.277993,0.010000,0.010000,0.010000,0.010000,0.277993,0.017261,0.010000
1.2.826.0.1.3680043.10005,0.909159,0.081598,0.023682,0.036645,0.023904,0.112568,0.909159,0.263305
1.2.826.0.1.3680043.10014,0.939342,0.010000,0.010000,0.010000,0.875282,0.939342,0.014269,0.010000
1.2.826.0.1.3680043.10016,0.554091,0.390641,0.062937,0.010000,0.010000,0.011762,0.042546,0.554091
1.2.826.0.1.3680043.10032,0.398840,0.280659,0.042830,0.010000,0.010000,0.367138,0.039941,0.398840
...,...,...,...,...,...,...,...,...
1.2.826.0.1.3680043.9926,0.970284,0.060180,0.970284,0.022860,0.153319,0.010000,0.010000,0.037407
1.2.826.0.1.3680043.9940,0.983749,0.078357,0.010000,0.010000,0.983749,0.047424,0.022098,0.022042
1.2.826.0.1.3680043.9994,0.716757,0.264916,0.023961,0.105340,0.010000,0.010000,0.032360,0.716757


In [19]:
df_effnet_pred.to_csv(os.path.join(DATA_DIR, 'predictions', f'df_effnet_pred_{task_code}.csv'))
df_test_pred.to_csv(os.path.join(DATA_DIR, 'predictions', f'df_test_pred_{task_code}.csv'))
df_patient_pred.to_csv(os.path.join(DATA_DIR,'predictions', f'df_patient_pred_{task_code}.csv'))

In [20]:
prob = df_patient_pred.values
label = train_df.loc[df_patient_pred.index].values

# print(prob.shape)
# print(label.shape)

losses = cal_loss(prob, label)
# print(list(losses))
# print(losses)
print(np.mean(losses))
# list(losses)

[0.190958818753383, 1.4131978500102247, 1.7540206455492549, 0.6898007587191056, 0.35447902884334326, 0.4587682359851897, 0.0540793663398786, 0.6867599468013006, 0.2086096877631332, 1.1087810980918056, 0.13872326132176177, 0.29615098737518897, 0.10550991033336946, 0.18425342744297307, 1.4943472140335612, 0.5190490713828931, 0.17212431372276374, 0.532036861607238, 0.1908871009280639, 0.11119803586708647, 0.23567967375063084, 0.4624250967648219, 0.38525516692210326, 0.7369320035871604, 0.2748976556717285, 0.20560444980351764, 0.3602513017725538, 1.3989940938823249, 0.010050325654447079, 0.12108367812735113, 1.25739565112495, 0.92388257663697, 0.06119263278586524, 0.06467332288792188, 0.4714979422092438, 0.17317735166712242, 0.4233919166560684, 0.010050325654447079, 0.037275058136362095, 0.5225040383890949, 1.2533642876812296, 0.08559830500079053, 0.020469158034150798, 0.20604497447077716, 0.48530778335407376, 0.0393902831710875, 0.47966180582131657, 0.04155882940228496, 0.0678647199113454

In [21]:
df_patient_pred['losses'] = losses
df_patient_pred.to_csv(os.path.join(DATA_DIR,'predictions', f'df_patient_loss_{task_code}.csv'))